In [1]:
from IPython.core.display import display, HTML

In [2]:
import salabim as sim
import sqlite3 as sql
import pandas as pd
import random
import arrow
from time import time
pd.set_option('display.max_columns', None)

In [63]:
blood_donor = blood_booking_data.iterrows()
plasma_donor = plasma_booking_data.iterrows()

def Normal(a, b):
    return sim.Bounded(sim.Normal(a, b), lowerbound=0).sample()

def Uniform(a, b):
    return sim.Uniform(a, b).sample()

class BloodDonor():
    def __init__(self):
        self.arrival = env.now()
        self.show = Uniform(0,1) < 0.6901
        self.rejected = Uniform(0,1) < 0.0147
        self.failed =  Uniform(0,1) < 0.0545
        self.declaration = Normal(5,2)
        self.reception = Normal(3,1)
        self.preperation = Normal(2,0.5)
        self.collection = Normal(8.125230202578269, 4.17646751840282) if not self.failed else Normal(8.576923076923077, 2.7057357441479324)
        self.check = Normal(1,0.2)
        self.rest = Normal(10,2)
        self.clean = Normal(1,0.2)
        self.type = rejected_blood_donors if self.rejected else failed_blood_donors if self.failed else blood_donors
        self.chair_type = blood_chairs
        
        self.wait = {}
        self.timestamps = {}
        
        bloods.append(self)
        
class PlasmaDonor():
    def __init__(self):
        self.arrival = env.now()
        self.show = Uniform(0,1) < 0.7152
        self.rejected = Uniform(0,1) < 0.0048
        self.failed = Uniform(0,1) < 0.0281
        self.declaration = Normal(5,2)
        self.reception = Normal(3,1)
        self.preperation = Normal(2,0.5)
        self.collection = Normal(33.95558775892933, 12.96371026189495) if not self.failed else Normal(9.38888888888889, 7.003746792667655)
        self.check = Normal(1,0.2)
        self.rest = Normal(1,0.2)
        self.clean = Normal(4,1)
        self.type = rejected_plasma_donors if self.rejected else failed_plasma_donors if self.failed else plasma_donors
        self.chair_type = plasma_chairs
        
        self.wait = {}
        self.timestamps = {}
        
        plasmas.append(self)

class DonorProcess(sim.Component):
    def __init__(self, donor):
            super(DonorProcess, self).__init__()
            self.donor = donor
    
    def process(self):

        donor = self.donor
        
        if donor.show:
            
            before = env.now()
            
            now = arrow.now().replace(hour=8, minute=0, second=0, microsecond=0)
            arrival = now.shift(minutes=+donor.arrival)
            donor.timestamps['arrival'] = arrival.format('HH:mm:ss')
            
            before_declaration = env.now()
            yield self.request(donor.type, declarations)
            donor.wait['declaration'] = env.now() - before_declaration
            yield self.hold(donor.declaration)
            self.release(declarations)

            before_reception = env.now()
            yield self.request(receptions, staff)
            donor.wait['reception'] = env.now() - before_reception
            yield self.hold(donor.reception)
            self.release(receptions, staff)

            if donor.rejected:

                self.release(donor.type)

            else:

                before_preperation = env.now()
                yield self.request(staff, donor.chair_type)
                donor.wait['preperation'] = env.now() - before_preperation
                yield self.hold(donor.preperation)
                self.release(staff)

                yield self.hold(donor.collection)

                before_check = env.now()
                yield self.request(staff)
                donor.wait['check'] = env.now() - before_check
                yield self.hold(donor.check)
                self.release(staff)

                yield self.hold(donor.rest)
                self.release(donor.type)

                before_clean = env.now()
                yield self.request(staff)
                donor.wait['clean'] = env.now() - before_clean
                yield self.hold(donor.clean)
                self.release(staff, donor.chair_type)
                
            done = env.now() - before
            
            leaving = now.shift(minutes=+donor.arrival + done)
            donor.timestamps['leaving'] = leaving.format('HH:mm:ss')
            donor.timestamps['duration'] = str(leaving - arrival)
            
            df1.loc[len(bloods) - 1] = [donor.show, donor.rejected, donor.failed, donor.timestamps['arrival'], donor.timestamps['leaving']]
            

        
class BloodDonorGenerator(sim.Component):
    def process(self):
        while True:
            yield self.hold(Normal(10,5))
            #yield self.hold(1)
            donor = BloodDonor()
            DonorProcess(donor)

class PlasmaDonorGenerator(sim.Component):
    def process(self):
        while True:
            yield self.hold(Normal(10,5))
            #yield self.hold(1)
            donor = PlasmaDonor()
            DonorProcess(donor)
            
class PrintStats(sim.Component):
    def process(self):
        while True:
            yield self.hold(1)
            print(env.now(), blood_chairs.claimed_quantity(), plasma_chairs.claimed_quantity())

print('-----------------------------------')
print()
            
for i in range(5):
            
    df1 = pd.DataFrame(columns=['show', 'rejected', 'failed', 'arrival', 'leaving'])
    df2 = pd.DataFrame(columns=['show', 'rejected', 'failed'])

    env = sim.Environment(time_unit='minutes', trace=False, random_seed='*')

    BloodDonorGenerator()
    PlasmaDonorGenerator()
    #PrintStats()

    blood_donors = sim.Resource("blood_donors", float('inf'))
    plasma_donors = sim.Resource("plasma_donors", float('inf'))
    rejected_blood_donors = sim.Resource("rejected_blood_donors", float('inf'))
    rejected_plasma_donors = sim.Resource("rejected_plasma_donors", float('inf'))
    failed_blood_donors = sim.Resource("failed_blood_donors", float('inf'))
    failed_plasma_donors = sim.Resource("failed_plasma_donors", float('inf'))
    declarations = sim.Resource("declarations", 2)
    receptions = sim.Resource("receptions", 2)
    blood_chairs = sim.Resource("blood_chairs", 4)
    plasma_chairs = sim.Resource("plasma_chairs", 7)
    staff = sim.Resource("staff", 4)


    env.run(duration=600)

    print('Blood')
    print('Complete\t', blood_donors.claimers().length_of_stay.mean())
    print('Rejected\t', rejected_blood_donors.claimers().length_of_stay.mean())
    print('Failed\t\t', failed_blood_donors.claimers().length_of_stay.mean())
    print()
    print('Plasma')
    print('Complete\t', plasma_donors.claimers().length_of_stay.mean())
    print('Rejected\t', rejected_plasma_donors.claimers().length_of_stay.mean())
    print('Failed\t\t', failed_plasma_donors.claimers().length_of_stay.mean())

    print()
    print('-----------------------------------')
    print()

-----------------------------------

Blood
Complete	 28.298204652128266
Rejected	 nan
Failed		 25.674598594771112

Plasma
Complete	 44.41614384103211
Rejected	 nan
Failed		 21.24524773527217

-----------------------------------

Blood
Complete	 29.84621442097678
Rejected	 7.072583932818269
Failed		 23.422942737865498

Plasma
Complete	 46.60714431200604
Rejected	 nan
Failed		 30.04941766183572

-----------------------------------

Blood
Complete	 29.911708621058086
Rejected	 nan
Failed		 28.425510019241944

Plasma
Complete	 49.16766003491528
Rejected	 nan
Failed		 nan

-----------------------------------

Blood
Complete	 29.522556392203686
Rejected	 nan
Failed		 30.180142843311927

Plasma
Complete	 47.86296682786298
Rejected	 nan
Failed		 26.126508964823223

-----------------------------------

Blood
Complete	 30.656391585155315
Rejected	 nan
Failed		 19.471991788126957

Plasma
Complete	 46.287027694001964
Rejected	 nan
Failed		 17.889184269820817

-----------------------------------



In [64]:
vars(bloods[13])

{'arrival': 144.31434367315148,
 'show': True,
 'rejected': False,
 'failed': False,
 'declaration': 5.5309045455672825,
 'reception': 3.078691508172544,
 'preperation': 1.4376702516138355,
 'collection': 11.752083320946973,
 'check': 0.7991674116516918,
 'rest': 12.370874030642511,
 'clean': 1.0520479977560626,
 'type': Resource (blood_donors),
 'chair_type': Resource (blood_chairs),
 'wait': {'declaration': 0.0,
  'reception': 0.0,
  'preperation': 0.0,
  'check': 0.0,
  'clean': 0.0},
 'timestamps': {'arrival': '10:24:18',
  'leaving': '11:00:20',
  'duration': '0:36:01.286344'}}

In [65]:
df1

,show,rejected,failed,arrival,leaving
2383,True,False,False,08:13:26,08:49:49
2384,True,False,False,08:32:04,09:01:22
2386,True,False,False,08:54:22,09:21:28
2388,True,False,False,09:03:58,09:48:21
2389,True,False,False,09:25:55,10:00:58
2390,True,False,False,09:12:08,10:12:34
2392,True,False,False,09:52:07,10:32:38
2394,True,False,False,10:26:41,10:55:05
2395,True,False,False,10:17:33,11:05:55
2396,True,False,False,10:49:41,11:20:43


In [69]:
blood_booking_data = pd.read_sql_query("SELECT CPR_HASH as CPR, BOOKING_RESULTAT as booking_result, MODTAGE_TID AS meeting_time, OPRINDELIG_TIMESLOT_START AS start, OPRINDELIG_TIMESLOT_SLUT AS end, KALDEKODE as donation_type, BOOKING_KODE AS booking_type FROM Export_Worksheet WHERE TAPPESTED LIKE '%3501%' AND TAPPE_DATO = 20180423 AND donation_type = '0-Blood' ORDER BY start ASC", bookings)

In [70]:
blood_collection_data = pd.read_sql_query("SELECT CPR_HASH as CPR, TID_MODTAGET as meeting_time, KALDEKODE as donation_type, BOOKING_KODE as booking_type FROM Export_Worksheet WHERE TAPPESTED LIKE '%3501%' AND DATO_TAPNING = '2018-04-23' AND donation_type = '0-Blood' ORDER BY BOOKING_TIMESLOT_START ASC", collections)

In [71]:
blood_collection_data

,CPR,meeting_time,donation_type,booking_type
0,3038930063,2018-04-23 08:01,0-Blood,M-Manual
1,1236885236,2018-04-23 08:07,0-Blood,E
2,2859484115,2018-04-23 08:24,0-Blood,M-Manual
3,3986528184,2018-04-23 08:49,0-Blood,M-Manual
4,1695867198,2018-04-23 09:07,0-Blood,M-Manual
5,2874812420,2018-04-23 09:13,0-Blood,M-Manual
6,3633004708,2018-04-23 10:14,0-Blood,M-Manual
7,2706068604,2018-04-23 10:31,0-Blood,M-Manual
8,196149938,2018-04-23 10:53,0-Blood,M-Manual
9,1168173398,2018-04-23 11:00,0-Blood,M-Manual


In [72]:
blood_booking_data

,CPR,booking_result,meeting_time,start,end,donation_type,booking_type
0,3038930063,3-Mødt,801,750,800,0-Blood,M-Manuelt
1,1236885236,3-Mødt,807,800,810,0-Blood,E
2,2859484115,3-Mødt,824,820,830,0-Blood,M-Manuelt
3,3440670012,2,0,820,830,0-Blood,M-Manuelt
4,3986528184,3-Mødt,849,840,850,0-Blood,M-Manuelt
5,1695867198,3-Mødt,907,900,910,0-Blood,M-Manuelt
6,2874812420,3-Mødt,913,910,920,0-Blood,M-Manuelt
7,3693475772,1-Ej mødt,909,910,920,0-Blood,E
8,3633004708,3-Mødt,1014,1010,1020,0-Blood,M-Manuelt
9,2706068604,3-Mødt,1031,1020,1030,0-Blood,M-Manuelt


In [65]:
plasma_booking_data

,booking_result,meeting_time,start,end,donation_type,booking_type
0,3-Mødt,800,755,800,1-Plasma,M-Manuelt
1,3-Mødt,803,755,800,1-Plasma,M-Manuelt
2,3-Mødt,814,800,805,1-Plasma,M-Manuelt
3,1-Ej mødt,912,805,810,1-Plasma,M-Manuelt
4,2,0,810,815,1-Plasma,M-Manuelt
5,3-Mødt,815,810,820,1-Plasma,M-Manuelt
6,3-Mødt,823,815,820,1-Plasma,M-Manuelt
7,1-Ej mødt,1537,840,850,1-Plasma,M-Manuelt
8,1-Ej mødt,2115,845,850,1-Plasma,E
9,3-Mødt,850,845,850,1-Plasma,M-Manuelt
